In [11]:
from snowflake.snowpark.session import Session

connection_params = {'user': "REFRACT.FOSFOR@LNTINFOTECH.COM", 'password': "Password321", 'account':"fya62509.us-east-1", 'role':"FOSFOR_REFRACT", 'warehouse': "FOSFOR_REFRACT"}

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

#data = session.sql("SHOW TABLES;").collect()
#print(data)

#data= session.sql("SELECT * from BIAS.BIASED_TABLE limit 1;").collect()
#print(data)

session.sql('CREATE OR REPLACE TABLE BIAS.sample_product_data (id INT, parent_id INT, category_id INT, name VARCHAR, serial_number VARCHAR, key INT, "3rd" INT);').collect()
r1=session.sql("""
    INSERT INTO BIAS.sample_product_data VALUES
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50)
    """).collect()
print(r1)

session.close()

[Row(number of rows inserted=5)]


In [12]:
from snowflake.snowpark.functions import col

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

df_table = session.table("BIAS.sample_product_data")
df_table1 = session.table("BIAS.sample_product_data").select(col("id"), col("name"), col("serial_number"))
#df1 = session.create_dataframe([1, 2, 3, 4]).to_df("a")
df_table.show()
df_table1.show()


session.close()

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
-------------------------------------------------------------------------------------

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
|3     |Product 1B  |prod

In [23]:
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.functions import udf

session = Session.builder.configs(connection_params).create()
session.sql("USE FOSFOR_REFRACT;").collect()

_ = session.sql("create or replace temp stage BIAS.mystage;").collect()
_ = session.udf.register(
    lambda x, y: x * y, return_type=IntegerType(),
    input_types=[IntegerType(), IntegerType()],
    is_permanent=True, name="mul", replace=True,
    stage_location="@mystage",
)
session.sql("select mul(5, 6) as mul").collect()



Failed to execute query [queryID: 01b2f551-0604-64b4-0072-f3030e79742e] ls '@mystage'
090106 (22000): Cannot perform STAGE LS. This session does not have a current schema. Call 'USE SCHEMA', or use a qualified name.


SnowparkSQLException: (1304): 01b2f551-0604-64b4-0072-f3030e79742e: 090106 (22000): Cannot perform STAGE LS. This session does not have a current schema. Call 'USE SCHEMA', or use a qualified name.